In [1]:
import numpy as np #importing numpy
import pandas as pd #importing pandas
import PIL #importing pillow for image processing
from PIL import Image#importing image from pillow library
import skimage#importing skimage library
import os#importing os library
import time#importing time library

#Capture start time of the program
start_time = time.time()

In [2]:
from string import digits
#code to remove numbers in a string and assigning it to the variable
r_digits = str.maketrans('', '', digits)
#taking source data folder path to a varible
rawdata_path = 'D://DKIT//dissertation//Dataset_of_Tomato_Leaves//plantvillage//Preprocessed_data//no_bias//data//'
#changing current librbay to source data
os.chdir(rawdata_path)
#taking sub-directories to a list
dirs = os.listdir()
directories = []
diseases = []
#Taking directories into a list and extract disease name from folder name
for x in dirs:
    res = x.translate(r_digits).lower()
    diseases.append(res)
    temp = rawdata_path + x + '//'
    directories.append(temp)

In [3]:
#creating different dataframes for saving data in different vector forms
#gray data
gray_df = pd.DataFrame(columns=['mean','disease'])
#rgb data
rgb_df = pd.DataFrame(columns=['mean','disease'])
#red data
red_df = pd.DataFrame(columns=['mean','disease'])
#green data
green_df = pd.DataFrame(columns=['mean','disease'])
#blue data
blue_df = pd.DataFrame(columns=['mean','disease'])

In [4]:
#extracting data from images
x = 0
#creating enpty directories to save the vector from in to a list
gray_test = []
rgb_final = []
red_final = []
green_final = []
blue_final = []
disease = []
#reading data directory by directory
for i in directories:
    os.chdir(i)
    k = diseases[x]
    pics = os.listdir()
    #reading pics one by one in each directory
    for j in pics:
        temp_list = []
        rgb_temp = []
        red_temp = []
        green_temp = []
        blue_temp = []
        gray = []
        data = []
        red = []
        green = []
        blue = []
        #opening the image in rgb format
        image = Image.open(j,mode='r')
        data = image.convert('RGB')
        data = np.array(data)
        #extracting only red components
        red = data[:,:,0]
        #extracting only green components
        green = data[:,:,1]
        #extracting only blue components
        blue = data[:,:,2]
        data = data.ravel()
        #saving rgb vector data into list
        rgb_final.append(data)
        rgb_temp.append(np.mean(data))
        rgb_temp.append(k)
        rgb_df.loc[len(rgb_df)] = rgb_temp
        #saving red channel data into dataframe and vector data to a list.
        red = red.ravel()
        red_final.append(red)
        red_temp.append(np.mean(red))
        red_temp.append(k)
        red_df.loc[len(red_df)] = red_temp
        #saving blue channel data into dataframe and vector data to a list.
        blue = blue.ravel()
        blue_final.append(blue)
        blue_temp.append(np.mean(blue))
        blue_temp.append(k)
        blue_df.loc[len(blue_df)] = blue_temp
        #saving green channel data into dataframe and vector data to a list.
        green = green.ravel()
        green_final.append(green)
        green_temp.append(np.mean(green))
        green_temp.append(k)
        green_df.loc[len(green_df)] = green_temp
        #opening the image in gray format
        gray = image.convert('L')
        gray = np.array(gray)
        gray = gray.ravel()
        gray_test.append(gray)
        temp_list.append(np.mean(gray))
        temp_list.append(k)       
        gray_df.loc[len(gray_df)] = temp_list
    x = x+1

In [5]:
#changing path
os.chdir('D://DKIT//dissertation//Dataset_of_Tomato_Leaves//plantvillage//Preprocessed_data//no_bias//vector//')

In [6]:
#saving the lists as numpy arrays in the local system
np.save('gray_test',gray_test)
np.save('rgb_data',rgb_final)
np.save('red_data',red_final)
np.save('green_data',green_final)
np.save('blue_data',blue_final)

In [7]:
#saving the dataframe as csv files in the local system
gray_df.to_csv('gray_data.csv',index=False)
rgb_df.to_csv('rgb_data.csv',index=False)
red_df.to_csv('red_data.csv',index=False)
green_df.to_csv('green_data.csv',index=False)
blue_df.to_csv('blue_data.csv',index=False)

# KNN on rgb data

In [8]:
#reading numpy array data
rgb_test = np.load('rgb_data.npy')
#reading csv file
rgb_df = pd.read_csv('rgb_data.csv')

In [9]:
loaded = []
for i in rgb_test:
    k = []
    k = np.asarray(i,dtype='uint8')
    loaded.append(k)

In [10]:
#assigning features and target
features = loaded
target = rgb_df['disease']
#splitting the data to 80 percent and 20 percent
from sklearn.model_selection import train_test_split
Ftrain,Ftest,Ltrain,Ltest=train_test_split(features,target, test_size=0.20, random_state=24)
#scaling the data using standard scaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
Ftrain = scaler.fit_transform(Ftrain)
Ftest = scaler.fit_transform(Ftest)

In [11]:
#K-Model on the data
from sklearn.neighbors import KNeighborsClassifier
kmodel = KNeighborsClassifier(n_neighbors=75)

In [12]:
#k-model fitting and accuracy
kmodel.fit(Ftrain,Ltrain)
print(kmodel.score(Ftest,Ltest))

0.38333333333333336


In [13]:
#importing classification report and confusion matrix
from sklearn.metrics import classification_report, confusion_matrix

In [14]:
#printing results of the kmodel
y_predict = kmodel.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.20      0.77      0.32       115
                 early_blight       1.00      0.01      0.02       120
                      healthy       0.36      0.66      0.46       125
                  late_blight       0.34      0.30      0.32       125
                    leaf_mold       0.69      0.58      0.63       127
           septoria_leaf_spot       0.64      0.32      0.42       117
                  target_spot       0.58      0.12      0.20       115
          tomato_mosaic_virus       0.67      0.04      0.07        54
tomato_yellow_leaf_curl_virus       0.75      0.42      0.54       114
      two-spotted_spider_mite       0.50      0.41      0.45       128

                     accuracy                           0.38      1140
                    macro avg       0.57      0.36      0.34      1140
                 weighted avg       0.57      0.38      0.36      1140

[[8

# Logistic on rgb data

In [15]:
#importing logistic data
from sklearn.linear_model import LogisticRegression
model=LogisticRegression(solver='lbfgs', max_iter=10000)
model.fit(Ftrain,Ltrain)
print(model.score(Ftest,Ltest))

0.6657894736842105


In [16]:
#classification report
y_predict = model.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.54      0.74      0.63       115
                 early_blight       0.51      0.40      0.45       120
                      healthy       0.83      0.60      0.70       125
                  late_blight       0.62      0.64      0.63       125
                    leaf_mold       0.75      0.77      0.76       127
           septoria_leaf_spot       0.64      0.63      0.64       117
                  target_spot       0.59      0.63      0.61       115
          tomato_mosaic_virus       0.76      0.87      0.81        54
tomato_yellow_leaf_curl_virus       0.82      0.86      0.84       114
      two-spotted_spider_mite       0.68      0.63      0.65       128

                     accuracy                           0.67      1140
                    macro avg       0.67      0.68      0.67      1140
                 weighted avg       0.67      0.67      0.66      1140

[[8

# SVM on RGB data

In [17]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

clf = SVC(kernel='linear')
clf.fit(Ftrain,Ltrain)
y_pred = clf.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.7157894736842105


In [18]:
#classification report
y_predict = clf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.64      0.83      0.72       115
                 early_blight       0.54      0.45      0.49       120
                      healthy       0.88      0.79      0.84       125
                  late_blight       0.65      0.69      0.67       125
                    leaf_mold       0.80      0.76      0.78       127
           septoria_leaf_spot       0.68      0.65      0.66       117
                  target_spot       0.72      0.72      0.72       115
          tomato_mosaic_virus       0.79      0.93      0.85        54
tomato_yellow_leaf_curl_virus       0.84      0.85      0.85       114
      two-spotted_spider_mite       0.66      0.62      0.64       128

                     accuracy                           0.72      1140
                    macro avg       0.72      0.73      0.72      1140
                 weighted avg       0.72      0.72      0.71      1140

[[9

# Random Forest on RGB 

In [19]:
from sklearn.ensemble import RandomForestClassifier

In [20]:
classifier_rf = RandomForestClassifier(random_state=42, n_jobs=-1, max_depth=5,n_estimators=100, oob_score=True)

In [21]:
classifier_rf.fit(Ftrain, Ltrain)

RandomForestClassifier(max_depth=5, n_jobs=-1, oob_score=True, random_state=42)

In [22]:
classifier_rf.oob_score_

0.4926518973459092

In [23]:
y_predict = classifier_rf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.46      0.68      0.55       115
                 early_blight       0.54      0.12      0.19       120
                      healthy       0.55      0.59      0.57       125
                  late_blight       0.58      0.39      0.47       125
                    leaf_mold       0.62      0.62      0.62       127
           septoria_leaf_spot       0.44      0.47      0.45       117
                  target_spot       0.38      0.57      0.46       115
          tomato_mosaic_virus       0.00      0.00      0.00        54
tomato_yellow_leaf_curl_virus       0.49      0.80      0.61       114
      two-spotted_spider_mite       0.49      0.43      0.46       128

                     accuracy                           0.49      1140
                    macro avg       0.45      0.47      0.44      1140
                 weighted avg       0.48      0.49      0.46      1140

[[7

C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Naive Bayes on RGB data

In [24]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(Ftrain, Ltrain)

GaussianNB()

In [25]:
y_pred = gnb.predict(Ftest)

In [26]:
from sklearn import metrics
metrics.accuracy_score(Ltest, y_pred)

0.437719298245614

In [27]:
class_names = target.unique()
print(classification_report(Ltest, y_pred, target_names=class_names))
print(confusion_matrix(Ltest, y_pred))

                               precision    recall  f1-score   support

               bacterial_spot       0.50      0.71      0.59       115
                 early_blight       0.32      0.29      0.31       120
                      healthy       0.38      0.49      0.43       125
                  late_blight       0.29      0.22      0.25       125
                    leaf_mold       0.70      0.40      0.51       127
           septoria_leaf_spot       0.40      0.41      0.40       117
                  target_spot       0.45      0.41      0.43       115
          tomato_mosaic_virus       0.35      0.69      0.46        54
tomato_yellow_leaf_curl_virus       0.64      0.45      0.53       114
      two-spotted_spider_mite       0.46      0.47      0.46       128

                     accuracy                           0.44      1140
                    macro avg       0.45      0.45      0.44      1140
                 weighted avg       0.45      0.44      0.43      1140

[[8

# Decision Tree on RGB data

In [28]:
from sklearn.tree import DecisionTreeClassifier
classifier_tree = DecisionTreeClassifier()

In [29]:
classifier_tree.fit(Ftrain, Ltrain)
classifier_tree.score(Ftest,Ltest)

0.3587719298245614

In [30]:
y_predict = classifier_tree.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.43      0.48      0.45       115
                 early_blight       0.20      0.23      0.22       120
                      healthy       0.49      0.37      0.42       125
                  late_blight       0.37      0.41      0.39       125
                    leaf_mold       0.47      0.48      0.47       127
           septoria_leaf_spot       0.33      0.32      0.32       117
                  target_spot       0.23      0.23      0.23       115
          tomato_mosaic_virus       0.20      0.19      0.19        54
tomato_yellow_leaf_curl_virus       0.44      0.53      0.48       114
      two-spotted_spider_mite       0.40      0.31      0.35       128

                     accuracy                           0.36      1140
                    macro avg       0.36      0.35      0.35      1140
                 weighted avg       0.37      0.36      0.36      1140

[[5

# KNN on grayscale data

In [31]:
#reading numpy array data
gray_test = np.load('gray_test.npy')
#reading csv file
gray_df = pd.read_csv('gray_data.csv')

In [32]:
loaded = []
for i in gray_test:
    k = []
    k = np.asarray(i,dtype='uint8')
    loaded.append(k)

In [33]:
features = loaded
target = gray_df['disease']
Ftrain,Ftest,Ltrain,Ltest=train_test_split(features,target, test_size=0.20, random_state=20)
Ftrain = scaler.fit_transform(Ftrain)
Ftest = scaler.fit_transform(Ftest)

In [34]:
kmodel.fit(Ftrain,Ltrain)
kmodel.score(Ftest,Ltest)

0.3192982456140351

In [35]:
y_predict = kmodel.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.22      0.74      0.35       124
                 early_blight       0.00      0.00      0.00       114
                      healthy       0.39      0.60      0.47       131
                  late_blight       0.25      0.31      0.28       124
                    leaf_mold       0.41      0.50      0.45       117
           septoria_leaf_spot       0.47      0.39      0.42       114
                  target_spot       0.22      0.04      0.06       111
          tomato_mosaic_virus       0.00      0.00      0.00        60
tomato_yellow_leaf_curl_virus       0.24      0.13      0.17       121
      two-spotted_spider_mite       0.59      0.27      0.37       124

                     accuracy                           0.32      1140
                    macro avg       0.28      0.30      0.26      1140
                 weighted avg       0.30      0.32      0.27      1140

[[9

C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Logistic on grayscale data

In [36]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression(solver='lbfgs', max_iter=10000)
model.fit(Ftrain,Ltrain)
model.score(Ftest,Ltest)

0.35

In [37]:
y_predict = model.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.34      0.45      0.39       124
                 early_blight       0.28      0.30      0.29       114
                      healthy       0.54      0.40      0.46       131
                  late_blight       0.24      0.31      0.27       124
                    leaf_mold       0.39      0.39      0.39       117
           septoria_leaf_spot       0.36      0.32      0.34       114
                  target_spot       0.29      0.20      0.24       111
          tomato_mosaic_virus       0.37      0.17      0.23        60
tomato_yellow_leaf_curl_virus       0.28      0.32      0.30       121
      two-spotted_spider_mite       0.50      0.52      0.51       124

                     accuracy                           0.35      1140
                    macro avg       0.36      0.34      0.34      1140
                 weighted avg       0.36      0.35      0.35      1140

[[5

# SVM on Grayscale data

In [38]:
clf = SVC(kernel='linear')
clf.fit(Ftrain,Ltrain)
y_pred = clf.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.4017543859649123


In [39]:
y_predict = clf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.34      0.52      0.42       124
                 early_blight       0.29      0.33      0.31       114
                      healthy       0.61      0.47      0.53       131
                  late_blight       0.32      0.35      0.33       124
                    leaf_mold       0.47      0.45      0.46       117
           septoria_leaf_spot       0.42      0.39      0.40       114
                  target_spot       0.31      0.26      0.28       111
          tomato_mosaic_virus       0.48      0.22      0.30        60
tomato_yellow_leaf_curl_virus       0.35      0.39      0.37       121
      two-spotted_spider_mite       0.58      0.52      0.55       124

                     accuracy                           0.40      1140
                    macro avg       0.42      0.39      0.40      1140
                 weighted avg       0.42      0.40      0.40      1140

[[6

# Random Forest on Grayscale

In [40]:
classifier_rf.fit(Ftrain, Ltrain)

RandomForestClassifier(max_depth=5, n_jobs=-1, oob_score=True, random_state=42)

In [41]:
classifier_rf.oob_score_

0.42070629524018427

In [42]:
y_predict = classifier_rf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.41      0.62      0.50       124
                 early_blight       0.34      0.15      0.21       114
                      healthy       0.49      0.53      0.51       131
                  late_blight       0.49      0.30      0.37       124
                    leaf_mold       0.46      0.55      0.50       117
           septoria_leaf_spot       0.53      0.57      0.55       114
                  target_spot       0.35      0.55      0.43       111
          tomato_mosaic_virus       0.00      0.00      0.00        60
tomato_yellow_leaf_curl_virus       0.33      0.26      0.29       121
      two-spotted_spider_mite       0.45      0.55      0.50       124

                     accuracy                           0.43      1140
                    macro avg       0.38      0.41      0.38      1140
                 weighted avg       0.41      0.43      0.41      1140

[[7

C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Naive bayes on Grayscale

In [43]:
gnb.fit(Ftrain, Ltrain)

GaussianNB()

In [44]:
y_pred = gnb.predict(Ftest)
metrics.accuracy_score(Ltest, y_pred)

0.393859649122807

In [45]:
class_names = target.unique()
print(classification_report(Ltest, y_pred, target_names=class_names))
print(confusion_matrix(Ltest, y_pred))

                               precision    recall  f1-score   support

               bacterial_spot       0.48      0.70      0.57       124
                 early_blight       0.35      0.32      0.33       114
                      healthy       0.43      0.55      0.48       131
                  late_blight       0.32      0.20      0.25       124
                    leaf_mold       0.62      0.30      0.40       117
           septoria_leaf_spot       0.41      0.48      0.44       114
                  target_spot       0.40      0.34      0.37       111
          tomato_mosaic_virus       0.25      0.55      0.34        60
tomato_yellow_leaf_curl_virus       0.28      0.11      0.15       121
      two-spotted_spider_mite       0.38      0.44      0.41       124

                     accuracy                           0.39      1140
                    macro avg       0.39      0.40      0.38      1140
                 weighted avg       0.40      0.39      0.38      1140

[[8

# Decision Tree on Grayscale

In [46]:
classifier_tree.fit(Ftrain, Ltrain)
classifier_tree.score(Ftest,Ltest)

0.26052631578947366

In [47]:
y_predict = classifier_tree.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.44      0.44      0.44       124
                 early_blight       0.17      0.19      0.18       114
                      healthy       0.36      0.27      0.31       131
                  late_blight       0.24      0.24      0.24       124
                    leaf_mold       0.31      0.32      0.31       117
           septoria_leaf_spot       0.26      0.29      0.27       114
                  target_spot       0.19      0.21      0.20       111
          tomato_mosaic_virus       0.14      0.10      0.12        60
tomato_yellow_leaf_curl_virus       0.16      0.19      0.18       121
      two-spotted_spider_mite       0.31      0.27      0.29       124

                     accuracy                           0.26      1140
                    macro avg       0.26      0.25      0.25      1140
                 weighted avg       0.27      0.26      0.26      1140

[[5

# KNN on Red channel data

In [48]:
#reading numpy array data
red_test = np.load('red_data.npy')
#reading csv file
red_df = pd.read_csv('red_data.csv')

In [49]:
loaded = []
for i in red_test:
    k = []
    k = np.asarray(i,dtype='uint8')
    loaded.append(k)
features = loaded
target = red_df['disease']
Ftrain,Ftest,Ltrain,Ltest=train_test_split(features,target, test_size=0.20, random_state=20)
scaler = StandardScaler()
Ftrain = scaler.fit_transform(Ftrain)
Ftest = scaler.fit_transform(Ftest)

In [50]:
kmodel.fit(Ftrain,Ltrain)
kmodel.score(Ftest,Ltest)

0.35175438596491226

In [51]:
y_predict = kmodel.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.29      0.72      0.41       124
                 early_blight       0.20      0.01      0.02       114
                      healthy       0.36      0.69      0.47       131
                  late_blight       0.27      0.27      0.27       124
                    leaf_mold       0.50      0.45      0.48       117
           septoria_leaf_spot       0.41      0.34      0.37       114
                  target_spot       0.23      0.10      0.14       111
          tomato_mosaic_virus       0.40      0.03      0.06        60
tomato_yellow_leaf_curl_virus       0.26      0.20      0.22       121
      two-spotted_spider_mite       0.57      0.47      0.51       124

                     accuracy                           0.35      1140
                    macro avg       0.35      0.33      0.30      1140
                 weighted avg       0.35      0.35      0.31      1140

[[8

# Logistic on Red channel data

In [52]:
model.fit(Ftrain,Ltrain)
model.score(Ftest,Ltest)

0.3631578947368421

In [53]:
y_predict = model.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.38      0.48      0.42       124
                 early_blight       0.26      0.28      0.27       114
                      healthy       0.56      0.42      0.48       131
                  late_blight       0.21      0.27      0.24       124
                    leaf_mold       0.43      0.44      0.44       117
           septoria_leaf_spot       0.34      0.31      0.32       114
                  target_spot       0.30      0.23      0.26       111
          tomato_mosaic_virus       0.39      0.23      0.29        60
tomato_yellow_leaf_curl_virus       0.32      0.35      0.33       121
      two-spotted_spider_mite       0.52      0.53      0.53       124

                     accuracy                           0.36      1140
                    macro avg       0.37      0.35      0.36      1140
                 weighted avg       0.37      0.36      0.36      1140

[[5

# SVM on Red channel data

In [54]:
clf.fit(Ftrain,Ltrain)
y_pred = clf.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.4166666666666667


In [55]:
y_predict = clf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.35      0.50      0.41       124
                 early_blight       0.26      0.27      0.27       114
                      healthy       0.63      0.47      0.54       131
                  late_blight       0.32      0.39      0.35       124
                    leaf_mold       0.48      0.44      0.46       117
           septoria_leaf_spot       0.36      0.33      0.35       114
                  target_spot       0.39      0.38      0.39       111
          tomato_mosaic_virus       0.62      0.30      0.40        60
tomato_yellow_leaf_curl_virus       0.41      0.42      0.41       121
      two-spotted_spider_mite       0.57      0.57      0.57       124

                     accuracy                           0.42      1140
                    macro avg       0.44      0.41      0.42      1140
                 weighted avg       0.43      0.42      0.42      1140

[[6

# Random forest on Red channel data

In [56]:
classifier_rf.fit(Ftrain, Ltrain)
classifier_rf.oob_score_

0.43474446150471596

In [57]:
y_predict = classifier_rf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.48      0.62      0.54       124
                 early_blight       0.33      0.16      0.21       114
                      healthy       0.57      0.62      0.60       131
                  late_blight       0.56      0.35      0.43       124
                    leaf_mold       0.47      0.57      0.51       117
           septoria_leaf_spot       0.45      0.44      0.45       114
                  target_spot       0.38      0.66      0.48       111
          tomato_mosaic_virus       0.00      0.00      0.00        60
tomato_yellow_leaf_curl_virus       0.32      0.26      0.29       121
      two-spotted_spider_mite       0.49      0.63      0.55       124

                     accuracy                           0.46      1140
                    macro avg       0.40      0.43      0.41      1140
                 weighted avg       0.43      0.46      0.43      1140

[[7

C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Naive Bayes on Red channel data

In [58]:
gnb.fit(Ftrain, Ltrain)

GaussianNB()

In [59]:
y_pred = gnb.predict(Ftest)
metrics.accuracy_score(Ltest, y_pred)

0.42719298245614035

In [60]:
class_names = target.unique()
print(classification_report(Ltest, y_pred, target_names=class_names))
print(confusion_matrix(Ltest, y_pred))

                               precision    recall  f1-score   support

               bacterial_spot       0.50      0.70      0.58       124
                 early_blight       0.26      0.21      0.23       114
                      healthy       0.43      0.61      0.50       131
                  late_blight       0.36      0.23      0.28       124
                    leaf_mold       0.60      0.38      0.46       117
           septoria_leaf_spot       0.38      0.41      0.39       114
                  target_spot       0.50      0.46      0.48       111
          tomato_mosaic_virus       0.35      0.55      0.43        60
tomato_yellow_leaf_curl_virus       0.38      0.15      0.21       121
      two-spotted_spider_mite       0.46      0.60      0.52       124

                     accuracy                           0.43      1140
                    macro avg       0.42      0.43      0.41      1140
                 weighted avg       0.42      0.43      0.41      1140

[[8

# Decision Tree on Red channel

In [61]:
classifier_tree.fit(Ftrain, Ltrain)
classifier_tree.score(Ftest,Ltest)

0.2868421052631579

In [62]:
y_predict = classifier_tree.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.37      0.39      0.38       124
                 early_blight       0.21      0.23      0.22       114
                      healthy       0.38      0.33      0.35       131
                  late_blight       0.35      0.40      0.38       124
                    leaf_mold       0.41      0.41      0.41       117
           septoria_leaf_spot       0.23      0.25      0.24       114
                  target_spot       0.25      0.25      0.25       111
          tomato_mosaic_virus       0.21      0.18      0.19        60
tomato_yellow_leaf_curl_virus       0.24      0.21      0.22       121
      two-spotted_spider_mite       0.32      0.31      0.32       124

                     accuracy                           0.30      1140
                    macro avg       0.30      0.30      0.30      1140
                 weighted avg       0.30      0.30      0.30      1140

[[4

# KNN on green channel data

In [63]:
#reading numpy array data
green_test = np.load('green_data.npy')
#reading csv file
green_df = pd.read_csv('green_data.csv')
loaded = []
for i in green_test:
    k = []
    k = np.asarray(i,dtype='uint8')
    loaded.append(k)
features = loaded
target = green_df['disease']
Ftrain,Ftest,Ltrain,Ltest=train_test_split(features,target, test_size=0.20, random_state=20)
scaler = StandardScaler()
Ftrain = scaler.fit_transform(Ftrain)
Ftest = scaler.fit_transform(Ftest)
kmodel.fit(Ftrain,Ltrain)
kmodel.score(Ftest,Ltest)

0.3157894736842105

In [64]:
y_predict = kmodel.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.21      0.76      0.32       124
                 early_blight       0.00      0.00      0.00       114
                      healthy       0.39      0.55      0.46       131
                  late_blight       0.25      0.30      0.27       124
                    leaf_mold       0.36      0.56      0.43       117
           septoria_leaf_spot       0.49      0.37      0.42       114
                  target_spot       0.43      0.03      0.05       111
          tomato_mosaic_virus       0.00      0.00      0.00        60
tomato_yellow_leaf_curl_virus       0.57      0.13      0.21       121
      two-spotted_spider_mite       0.67      0.25      0.36       124

                     accuracy                           0.32      1140
                    macro avg       0.34      0.29      0.25      1140
                 weighted avg       0.36      0.32      0.27      1140

[[9

C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Logistic on Green channel data

In [65]:
model.fit(Ftrain,Ltrain)
model.score(Ftest,Ltest)

0.34912280701754383

In [66]:
y_predict = model.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.32      0.45      0.38       124
                 early_blight       0.29      0.31      0.30       114
                      healthy       0.54      0.39      0.45       131
                  late_blight       0.26      0.32      0.29       124
                    leaf_mold       0.36      0.40      0.38       117
           septoria_leaf_spot       0.38      0.39      0.38       114
                  target_spot       0.27      0.18      0.22       111
          tomato_mosaic_virus       0.28      0.08      0.13        60
tomato_yellow_leaf_curl_virus       0.29      0.34      0.31       121
      two-spotted_spider_mite       0.51      0.48      0.49       124

                     accuracy                           0.35      1140
                    macro avg       0.35      0.33      0.33      1140
                 weighted avg       0.36      0.35      0.35      1140

[[5

# SVM on green channel data

In [67]:
clf.fit(Ftrain,Ltrain)
y_pred = clf.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.4052631578947368


In [68]:
y_predict = clf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.33      0.54      0.41       124
                 early_blight       0.27      0.32      0.29       114
                      healthy       0.62      0.47      0.54       131
                  late_blight       0.31      0.35      0.33       124
                    leaf_mold       0.51      0.50      0.51       117
           septoria_leaf_spot       0.40      0.40      0.40       114
                  target_spot       0.33      0.23      0.28       111
          tomato_mosaic_virus       0.53      0.17      0.25        60
tomato_yellow_leaf_curl_virus       0.40      0.45      0.42       121
      two-spotted_spider_mite       0.59      0.47      0.52       124

                     accuracy                           0.41      1140
                    macro avg       0.43      0.39      0.39      1140
                 weighted avg       0.43      0.41      0.40      1140

[[6

# Random Forest on green channel data

In [69]:
classifier_rf.fit(Ftrain, Ltrain)
classifier_rf.oob_score_

0.40908093880236895

In [70]:
y_predict = classifier_rf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.41      0.64      0.50       124
                 early_blight       0.26      0.18      0.21       114
                      healthy       0.48      0.56      0.52       131
                  late_blight       0.48      0.31      0.38       124
                    leaf_mold       0.39      0.48      0.43       117
           septoria_leaf_spot       0.49      0.59      0.53       114
                  target_spot       0.30      0.51      0.38       111
          tomato_mosaic_virus       0.00      0.00      0.00        60
tomato_yellow_leaf_curl_virus       0.53      0.22      0.31       121
      two-spotted_spider_mite       0.48      0.46      0.47       124

                     accuracy                           0.42      1140
                    macro avg       0.38      0.40      0.37      1140
                 weighted avg       0.41      0.42      0.40      1140

[[7

C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Naive bayes on Green channel data

In [71]:
gnb.fit(Ftrain, Ltrain)
y_pred = gnb.predict(Ftest)
metrics.accuracy_score(Ltest, y_pred)

0.37719298245614036

In [72]:
class_names = target.unique()
print(classification_report(Ltest, y_pred, target_names=class_names))
print(confusion_matrix(Ltest, y_pred))

                               precision    recall  f1-score   support

               bacterial_spot       0.48      0.69      0.57       124
                 early_blight       0.38      0.36      0.37       114
                      healthy       0.47      0.56      0.51       131
                  late_blight       0.28      0.19      0.23       124
                    leaf_mold       0.50      0.24      0.32       117
           septoria_leaf_spot       0.38      0.48      0.42       114
                  target_spot       0.42      0.32      0.36       111
          tomato_mosaic_virus       0.21      0.48      0.29        60
tomato_yellow_leaf_curl_virus       0.17      0.07      0.10       121
      two-spotted_spider_mite       0.36      0.41      0.38       124

                     accuracy                           0.38      1140
                    macro avg       0.36      0.38      0.36      1140
                 weighted avg       0.37      0.38      0.36      1140

[[8

# Decision Tree on Green channel

In [73]:
classifier_tree.fit(Ftrain, Ltrain)
classifier_tree.score(Ftest,Ltest)

0.2517543859649123

In [74]:
y_predict = classifier_tree.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.36      0.40      0.38       124
                 early_blight       0.17      0.18      0.17       114
                      healthy       0.47      0.42      0.44       131
                  late_blight       0.27      0.27      0.27       124
                    leaf_mold       0.31      0.37      0.34       117
           septoria_leaf_spot       0.33      0.30      0.31       114
                  target_spot       0.15      0.15      0.15       111
          tomato_mosaic_virus       0.08      0.08      0.08        60
tomato_yellow_leaf_curl_virus       0.21      0.20      0.20       121
      two-spotted_spider_mite       0.25      0.23      0.24       124

                     accuracy                           0.27      1140
                    macro avg       0.26      0.26      0.26      1140
                 weighted avg       0.27      0.27      0.27      1140

[[4

# KNN on blue channel data

In [75]:
#reading numpy array data
blue_test = np.load('blue_data.npy')
#reading csv file
blue_df = pd.read_csv('blue_data.csv')
loaded = []
for i in gray_test:
    k = []
    k = np.asarray(i,dtype='uint8')
    loaded.append(k)
features = loaded
target = blue_df['disease']
Ftrain,Ftest,Ltrain,Ltest=train_test_split(features,target, test_size=0.20, random_state=20)
Ftrain = scaler.fit_transform(Ftrain)
Ftest = scaler.fit_transform(Ftest)
kmodel.fit(Ftrain,Ltrain)
kmodel.score(Ftest,Ltest)

0.3192982456140351

In [76]:
y_predict = kmodel.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.22      0.74      0.35       124
                 early_blight       0.00      0.00      0.00       114
                      healthy       0.39      0.60      0.47       131
                  late_blight       0.25      0.31      0.28       124
                    leaf_mold       0.41      0.50      0.45       117
           septoria_leaf_spot       0.47      0.39      0.42       114
                  target_spot       0.22      0.04      0.06       111
          tomato_mosaic_virus       0.00      0.00      0.00        60
tomato_yellow_leaf_curl_virus       0.24      0.13      0.17       121
      two-spotted_spider_mite       0.59      0.27      0.37       124

                     accuracy                           0.32      1140
                    macro avg       0.28      0.30      0.26      1140
                 weighted avg       0.30      0.32      0.27      1140

[[9

C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# logistic on blue channel data

In [77]:
model.fit(Ftrain,Ltrain)
model.score(Ftest,Ltest)

0.35

In [78]:
y_predict = model.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.34      0.45      0.39       124
                 early_blight       0.28      0.30      0.29       114
                      healthy       0.54      0.40      0.46       131
                  late_blight       0.24      0.31      0.27       124
                    leaf_mold       0.39      0.39      0.39       117
           septoria_leaf_spot       0.36      0.32      0.34       114
                  target_spot       0.29      0.20      0.24       111
          tomato_mosaic_virus       0.37      0.17      0.23        60
tomato_yellow_leaf_curl_virus       0.28      0.32      0.30       121
      two-spotted_spider_mite       0.50      0.52      0.51       124

                     accuracy                           0.35      1140
                    macro avg       0.36      0.34      0.34      1140
                 weighted avg       0.36      0.35      0.35      1140

[[5

# SVM on blue channel data

In [79]:
clf.fit(Ftrain,Ltrain)
y_pred = clf.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.4017543859649123


In [80]:
y_predict = clf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.34      0.52      0.42       124
                 early_blight       0.29      0.33      0.31       114
                      healthy       0.61      0.47      0.53       131
                  late_blight       0.32      0.35      0.33       124
                    leaf_mold       0.47      0.45      0.46       117
           septoria_leaf_spot       0.42      0.39      0.40       114
                  target_spot       0.31      0.26      0.28       111
          tomato_mosaic_virus       0.48      0.22      0.30        60
tomato_yellow_leaf_curl_virus       0.35      0.39      0.37       121
      two-spotted_spider_mite       0.58      0.52      0.55       124

                     accuracy                           0.40      1140
                    macro avg       0.42      0.39      0.40      1140
                 weighted avg       0.42      0.40      0.40      1140

[[6

# Random Forest on blue channel data

In [81]:
classifier_rf.fit(Ftrain, Ltrain)
classifier_rf.oob_score_

0.42070629524018427

In [82]:
y_predict = classifier_rf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.41      0.62      0.50       124
                 early_blight       0.34      0.15      0.21       114
                      healthy       0.49      0.53      0.51       131
                  late_blight       0.49      0.30      0.37       124
                    leaf_mold       0.46      0.55      0.50       117
           septoria_leaf_spot       0.53      0.57      0.55       114
                  target_spot       0.35      0.55      0.43       111
          tomato_mosaic_virus       0.00      0.00      0.00        60
tomato_yellow_leaf_curl_virus       0.33      0.26      0.29       121
      two-spotted_spider_mite       0.45      0.55      0.50       124

                     accuracy                           0.43      1140
                    macro avg       0.38      0.41      0.38      1140
                 weighted avg       0.41      0.43      0.41      1140

[[7

C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Naive bayes on blue channel data

In [83]:
gnb.fit(Ftrain, Ltrain)
y_pred = gnb.predict(Ftest)
metrics.accuracy_score(Ltest, y_pred)

0.393859649122807

In [84]:
class_names = target.unique()
print(classification_report(Ltest, y_pred, target_names=class_names))
print(confusion_matrix(Ltest, y_pred))

                               precision    recall  f1-score   support

               bacterial_spot       0.48      0.70      0.57       124
                 early_blight       0.35      0.32      0.33       114
                      healthy       0.43      0.55      0.48       131
                  late_blight       0.32      0.20      0.25       124
                    leaf_mold       0.62      0.30      0.40       117
           septoria_leaf_spot       0.41      0.48      0.44       114
                  target_spot       0.40      0.34      0.37       111
          tomato_mosaic_virus       0.25      0.55      0.34        60
tomato_yellow_leaf_curl_virus       0.28      0.11      0.15       121
      two-spotted_spider_mite       0.38      0.44      0.41       124

                     accuracy                           0.39      1140
                    macro avg       0.39      0.40      0.38      1140
                 weighted avg       0.40      0.39      0.38      1140

[[8

# Decision Tree on Blue channel

In [85]:
classifier_tree.fit(Ftrain, Ltrain)
classifier_tree.score(Ftest,Ltest)

0.2771929824561403

In [86]:
y_predict = classifier_tree.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.41      0.42      0.42       124
                 early_blight       0.22      0.22      0.22       114
                      healthy       0.39      0.29      0.33       131
                  late_blight       0.25      0.27      0.26       124
                    leaf_mold       0.36      0.32      0.34       117
           septoria_leaf_spot       0.30      0.28      0.29       114
                  target_spot       0.21      0.22      0.21       111
          tomato_mosaic_virus       0.14      0.12      0.13        60
tomato_yellow_leaf_curl_virus       0.15      0.19      0.17       121
      two-spotted_spider_mite       0.32      0.37      0.34       124

                     accuracy                           0.28      1140
                    macro avg       0.28      0.27      0.27      1140
                 weighted avg       0.28      0.28      0.28      1140

[[5

In [87]:
end_time = time.time()
print(end_time - start_time)

32015.804406881332
